In [2]:
import os
import shutil
import cv2
import random
import matplotlib.pyplot as plt
import numpy as np
import copy
import wandb

In [15]:
! pip install ultralytics


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [4]:
from ultralytics import YOLO

In [5]:
datapath = '/Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/data/animals'
os.listdir(datapath)

['sparrow',
 'cat',
 'butterfly',
 '.DS_Store',
 'dog',
 'pigeon',
 'parrot',
 'squirrel',
 'cow']

In [6]:
image_size=640

In [9]:
# Creating paths for separate images and labels
curr_path=os.getcwd()
imgtrainpath = os.path.join(curr_path,'images','train')
imgvalpath=os.path.join(curr_path,'images','validation')
imgtestpath=os.path.join(curr_path,'images','test')

labeltrainpath=os.path.join(curr_path,'labels','train')
labelvalpath=os.path.join(curr_path,'labels','validation')
labeltestpath=os.path.join(curr_path,'labels','test')

# Creating directories for all paths defined
os.makedirs(imgtrainpath)
os.makedirs(imgvalpath)
os.makedirs(imgtestpath)
os.makedirs(labeltrainpath)
os.makedirs(labelvalpath)
os.makedirs(labeltestpath)

FileExistsError: [Errno 17] File exists: '/Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/images/train'

In [10]:
for dirname in os.listdir(datapath):
    dirpath=os.path.join(datapath, dirname)
    if os.path.isdir(dirpath):
        for file in os.listdir(dirpath):
            filepath=os.path.join(dirpath, file)
            newname=dirname+'_'+file
            if file.endswith((".txt")): # if label file, take it to label train path
                shutil.copy(filepath, labeltrainpath)
                path=os.path.join(labeltrainpath, file)
                newpath=os.path.join(labeltrainpath, newname)
            elif file.endswith((".jpg", ".JPG")): # if image file, resize and take it to image train path
                img_resized=cv2.resize(cv2.imread(filepath), (image_size, image_size))
                path=os.path.join(imgtrainpath, file)
                cv2.imwrite(path, img_resized)
                newpath=os.path.join(imgtrainpath, newname)
            os.rename(path, newpath) # Rename the file (label or image)
                

In [11]:
len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))

(480, 6)

In [12]:
def are_all_elements_same(lst):
    if not lst:
        return True  # An empty list is considered to have all elements the same.

    first_element = lst[0]
    for element in lst[1:]:
        if element != first_element:
            return False

    return True

In [13]:
for file in os.listdir(labeltrainpath):
    classes_list=[]
    with open(os.path.join(labeltrainpath, file), "r") as f:
        for line in f:
            class_id,_,_,_,_=line.strip().split(" ")
            classes_list.append(class_id) # creating list of all unique animal types in given image
            
    # Checking if different types of animals are present in image
    if not are_all_elements_same(classes_list): 
        filepath=os.path.join(labeltrainpath, file)
        newpath=os.path.join(labeltestpath, file)
        shutil.move(filepath, newpath) # moving label file to test path
        basename=os.path.splitext(file)[0]
        print(basename) # printing the image name
        imgfilename=basename+'.jpg'
        oldimgfilepath=os.path.join(imgtrainpath, imgfilename)
        newimgfilepath=os.path.join(imgtestpath, imgfilename) 
        shutil.move(oldimgfilepath, newimgfilepath) # moving image to test path

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [30]:
plt.figure(figsize=(30,30))
files = os.listdir(imgtestpath)
if len(files) >= 6:
    for i in range(6):
        test_image=os.path.join(imgtestpath, os.listdir(imgtestpath)[i])
        ax=plt.subplot(3,2,i+1)
    
    # Display actual image
        plt.imshow(cv2.imread(test_image)) 
        plt.xticks([])
        plt.yticks([])

<Figure size 3000x3000 with 0 Axes>

In [15]:
len(os.listdir(imgtestpath)), len(os.listdir(labeltestpath))

(0, 0)

In [16]:
len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))

(480, 6)

In [18]:
factor=0.2 

for file in random.sample(os.listdir(imgtrainpath), int(len(os.listdir(imgtrainpath))*factor)):
    basename=os.path.splitext(file)[0]
    textfilename=basename+'.jpg'
    labelfilepath=os.path.join(labeltrainpath, textfilename)
    labeldestpath=os.path.join(labelvalpath, textfilename)
    imgfilepath=os.path.join(imgtrainpath, file)
    imgdestpath=os.path.join(imgvalpath, file)
    if os.path.exists(labelfilepath):
    # Move the file
        shutil.move(labelfilepath, labeldestpath)
    else:
        print(f"The file '{labelfilepath}' does not exist.")
    shutil.move(imgfilepath, imgdestpath)
    shutil.move(labelfilepath, labeldestpath)
    

The file '/Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/train/cat_9fd544a838.jpg' does not exist.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/train/cat_9fd544a838.jpg'

In [19]:
len(os.listdir(imgtrainpath)), len(os.listdir(labeltrainpath))

(478, 6)

In [20]:
len(os.listdir(imgvalpath)), len(os.listdir(labelvalpath))

(8, 0)

In [21]:
def get_bbox_from_label(text_file_path):
    bbox_list=[]
    with open(text_file_path, "r") as file:
        for line in file:
            class_id,x_centre,y_centre,width,height=line.strip().split(" ")
            x1=(float(x_centre)+(float(width)/2))*image_size
            x0=(float(x_centre)-(float(width)/2))*image_size
            y1=(float(y_centre)+(float(height)/2))*image_size
            y0=(float(y_centre)-(float(height)/2))*image_size
            
            vertices=np.array([[int(x0), int(y0)], [int(x1), int(y0)], 
                               [int(x1),int(y1)], [int(x0),int(y1)]])
            bbox_list.append(vertices)      
    return tuple(bbox_list)

In [22]:
red=(255,0,0) 

In [23]:
newline='\n'

In [26]:
ln_1='# Train/val/test sets'+newline

# train, val and test path declaration
ln_2='train: ' +"'"+imgtrainpath+"'"+newline
ln_3='val: ' +"'" + imgvalpath+"'"+newline
ln_4='test: ' +"'" + imgtestpath+"'"+newline
ln_5=newline
ln_6='# Classes'+newline

# names of the classes declaration
ln_7='names:'+newline
ln_8='  0: buffalo'+newline
ln_9='  1: elephant'+newline
ln_10='  2: rhino'+newline
ln_11='  3: zebra'

config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8, ln_9, ln_10, ln_11]

In [27]:
config_path=os.path.join(curr_path, 'config.yaml')
config_path

'/Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/config.yaml'

In [28]:
with open(config_path, 'w') as f:
    f.writelines(config_lines)

In [29]:
model=YOLO('yolov8m.yaml').load('yolov8m.pt')


                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

In [30]:
results=model.train(data=config_path, epochs=100, iou=0.5, conf=0.01)

New https://pypi.org/project/ultralytics/8.1.2 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.1 🚀 Python-3.10.13 torch-2.1.2 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/config.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.01, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cl

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key f

Error: An unexpected error occurred

In [32]:
def evaluate_map50(trainedmodel, data_path, dataset='val'):
    metrics=trainedmodel.val(data=data_path, split=dataset)
    map50=round(metrics.box.map50, 3)
    print("The mAP of model for all images on {0} dataset is {1}".format(dataset,map50))
    return metrics, map50

In [33]:
def display_curves(root_path):
    plt.figure(figsize=(50,50))
    
    #displaying p curve
    p_curve=cv2.imread(os.path.join(root_path,'P_curve.png'))
    ax=plt.subplot(5,1,1)
    plt.imshow(p_curve)
    
    #displaying r curve
    r_curve=cv2.imread(os.path.join(root_path,'R_curve.png'))
    ax=plt.subplot(5,1,2)
    plt.imshow(r_curve)
    
    #displaying pr curve
    pr_curve=cv2.imread(os.path.join(root_path,'PR_curve.png'))
    ax=plt.subplot(5,1,3)
    plt.imshow(pr_curve)
    
    #displaying f1 curve
    f1_curve=cv2.imread(os.path.join(root_path,'F1_curve.png'))
    ax=plt.subplot(5,1,4)
    plt.imshow(f1_curve)
    
    #displaying confusion matrix
    confusion_matrix=cv2.imread(os.path.join(root_path,'confusion_matrix.png'))
    ax=plt.subplot(5,1,5)
    plt.imshow(confusion_matrix)

In [34]:
train_metrics, train_map50=evaluate_map50(model, config_path, dataset='train')

Ultralytics YOLOv8.1.1 🚀 Python-3.10.13 torch-2.1.2 CPU (Apple M2)
YOLOv8m summary (fused): 218 layers, 25842076 parameters, 14796 gradients, 78.7 GFLOPs


val: Scanning /Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/train... 0 images, 478 backgrounds, 0 corrupt: 100%|██████████| 478/478 [00:00<00:00, 10036.98it/s]

val: WARNING ⚠️ No labels found in /Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/train.cache. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.
val: New cache created: /Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/train.cache
WARNING ⚠️ No labels found in /Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [05:12<00:00, 10.43s/it]

                   all        478          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 0.8ms preprocess, 639.3ms inference, 0.0ms loss, 11.3ms postprocess per image
Results saved to runs/detect/val
The mAP of model for all images on train dataset is 0.0


In [35]:
train_path=os.path.join(curr_path, 'runs', 'detect', 'val')

In [37]:
val_metrics, val_map50=evaluate_map50(model, config_path, dataset='val')

Ultralytics YOLOv8.1.1 🚀 Python-3.10.13 torch-2.1.2 CPU (Apple M2)


val: Scanning /Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/validation... 0 images, 8 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<00:00, 6258.99it/s]

val: WARNING ⚠️ No labels found in /Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/validation.cache. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.
val: New cache created: /Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/validation.cache
WARNING ⚠️ No labels found in /Users/karpagapriyadhanraj/Desktop/EPITA/Computer vision/Computer_vision/Lab_2/aminal_detection/labels/validation.cache, training may not work correctly. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.61s/it]

                   all          8          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 1.4ms preprocess, 435.3ms inference, 0.0ms loss, 11.5ms postprocess per image
Results saved to runs/detect/val2
The mAP of model for all images on val dataset is 0.0


In [38]:
val_path=os.path.join(curr_path, 'runs', 'detect', 'val2') 

In [42]:
plt.figure(figsize=(30,30))
m=random.randint(0, 17) # Selecting random image number from 18 images in test dataset
files = os.listdir(imgtestpath)
if files:
    m = random.randint(0, len(files) - 1)
    for i in range(1,6,2):
        test_image=os.path.join(imgtestpath, os.listdir(imgtestpath)[m])
        ax=plt.subplot(3,2,i)
        
    # Display actual image
    plt.imshow(cv2.imread(test_image)) 
    plt.xticks([])
    plt.yticks([])
    plt.title("Actual image", fontsize = 40)
    
    # Predict 
    res = model(test_image)
    res_plotted = res[0].plot()
    ax=plt.subplot(3,2,i+1)
    
    # Display image with predictions
    plt.imshow(res_plotted)
    plt.title("Image with predictions", fontsize = 40)
    plt.xticks([])
    plt.yticks([])
    m=m+1

<Figure size 3000x3000 with 0 Axes>